# 对Word2Vec的一点理解

这一周仔细研究了一下自然语言处理方面（NLP）的一个大问题，Word2Vec。并且针对这个问题手动用代码实现了一下。所谓Word2Vec就是将自然语言中的词汇转化成向量形式。在传统的转化方案中大多是利用one-hot编码，也就是说若一个语料库中拥有三万个词汇，那么每一个词转化成的向量就会拥有三万维，整个语料库变成了一个巨大的稀疏矩阵，这对内存和计算力来说都是不小的挑战。后来随着神经网络的兴起，Word2Vec摆脱了以往傻大黑粗的转化方式，开始利用神经网络进行编码，当然这是另一个分枝了。今天笔者要讲的是Google提出的一种Word2Vec的方式，这种方式不但可以用有限的维数（通常不超过150维），而且还能够保留词与词之间的语义联系。话不多说，下面开始详细介绍。

## 介绍

由Google提出的这种Word2Vec模型是有着非常深厚数学基础的，在介绍这种方法之前首先我们应该先了解这个模型的两种应用。第一种是CBOW(Continuous Bag of Words)，另一种是Skip-Gram。这两种Word2Vec模型都是语义预测模型，数学推导非常相近，去用于解决两类截然不同的问题。CBOW用于解决通过上下文预测中心词的问题，也就是说我们给出一句话的上下文，要求该模型能够预测出中心词是什么。Skip-Gram模型则恰恰相反，它用于预测上下文，就是说我们给出一个中心词，要求模型能够给出与该中心词相关的上下文。在本文中笔者以Skip-Gram模型为基础介绍模型的一些细节和实现。

## 数学基础

Skip-Gram模型中已经给出了模型的目标，就是要根据中心词预测出上下文词汇。该模型是依据概率论中的观点，建立一个似然函数来实现模型的目标的。该模型的本质还是去解一个最优化问题的解，其优化问题的表达就是上文提到的那个似然函数。在Skip-Gram模型中待优化的目标函数可以写成如下形式：

$$G=\prod_{\omega \in C}^{ }\prod_{u \in Context(\omega )}^{ }g(u)$$


上式中w就是模型中传入的中心词，u是w的上下文中的词。该优化函数所要表达的意思就是。在整个语料库C的范围内，给出任意一个词w，预测出其上下文u的概率要最大化。接下来我们再给出g(u)的形式：

$$g(u)=\prod_{z\in \{u\}\cup NEG(u)}^{ }p(z|\omega )$$

上式中NEG(u)为负样本集合，何为负样本？考虑Skip-Gram模型的这样一个输入(w, context(x))，若该输入的标签不为context(w)，则我们就说这个样本为一个负样本。其中，p(z|w)的概率可以表达为如下形式：

$$p(z|\omega )=\left\{
\begin{aligned}
&x  =  \sigma (v(\omega )^{T}\theta ^{z}), L^{u}(z)=1\\
&y  =  1-\sigma (v(\omega )^{T}\theta ^{z}), L^{u}(z)=0
\end{aligned}
\right.$$

从上面的表达式可以看出该概率表达式将预测问题转化成了一个二分类问题，每个分类的结果用概率表示。当分类正确的概率表达为sigmod函数下中心词的向量表达乘以一个参数，该参数是未知的。而分类错误的概率恰恰为一减分类正确的概率。上面概率用一个式子可以表达为：

$$p(z|\omega )=[\sigma (v(\omega )^{T}\theta ^{z})]^{L^{u}(z)}[1-\sigma (v(\omega )^{T}\theta ^{z})]^{1-L^{u}(z)}
$$

则，我们可以对G取似然函数，也就是取log对数得到如下表达：

$$\begin{align*}
L=logG&=log\prod_{\omega \in C}^{ }\prod_{u \in Context(\omega )}^{ }\prod_{z\in \{u\}\cup NEG(u)}^{ }p(z|\omega )\\
&=\sum_{\omega \in C}^{ }\sum_{u \in Context(\omega )}^{ }\sum_{z\in \{u\}\cup NEG(u)}^{ }log\ p(z|\omega )\\
&=\sum_{\omega \in C}^{ }\sum_{u \in Context(\omega )}^{ }\sum_{z\in \{u\}\cup NEG(u)}^{ }log\ \{[\sigma (v(\omega )^{T}\theta ^{z})]^{L^{u}(z)}[1-\sigma (v(\omega )^{T}\theta ^{z})]^{1-L^{u}(z)}\}\\
&=\sum_{\omega \in C}^{ }\sum_{u \in Context(\omega )}^{ }\sum_{z\in \{u\}\cup NEG(u)}^{ }\{L^{u}(z)log\ [\sigma (v(\omega )^{T}\theta ^{z})](1-L^{u}(z))log\ [1-\sigma (v(\omega )^{T}\theta ^{z})]\}
\end{align*}$$

上式就是Skip-Gram模型的似然函数或者说是优化函数了，我们接下来只需要针对该似然函数关于v(w)和theta的偏导就能根据梯度上升法求解这个问题了。然而在实际中我们并不是使用这个似然函数作为优化目标的，因为从该似然函数的形式中也可以看出，该似然函数针对每个词w的上下文context(w)都要求|context(w)|次负样本，这样的计算量太大，因此，我们重新定义一个似然函数，形式如下：

$$\begin{align*}\
L=logG&=log\prod_{\omega \in C}^{ }\prod_{\tilde{\omega} \in Context(\omega )}^{ }\prod_{u\in \{\omega\}\cup NEG(\omega)}^{ }p(u|\tilde{\omega} )\\
&=\sum_{\omega \in C}^{ }\sum_{\tilde{\omega} \in Context(\omega )}^{ }\sum_{u\in \{\omega\}\cup NEG(\omega)}^{ }log\ p(u|\tilde{\omega} )\\
&=\sum_{\omega \in C}^{ }\sum_{\tilde{\omega} \in Context(\omega )}^{ }\sum_{u\in \{\omega\}\cup NEG(\omega)}^{ }log\ \{[\sigma (v(\tilde{\omega} )^{T}\theta ^{z})]^{L^{u}(z)}[1-\sigma (v(\tilde{\omega} )^{T}\theta ^{z})]^{1-L^{u}(z)}\}\\
&=\sum_{\omega \in C}^{ }\sum_{\tilde{\omega} \in Context(\omega )}^{ }\sum_{u\in \{\omega\}\cup NEG(\omega)}^{ }\{L^{u}(z)log\ [\sigma (v(\tilde{\omega} )^{T}\theta ^{z})](1-L^{u}(z))log\ [1-\sigma (v(\tilde{\omega} )^{T}\theta ^{z})]\}
\end{align*}$$

如上面的似然函数所示，现在针对一个context(w)只用一次负样本抽样就可以了。为了方便，我们将花括号内部的式子用一个标记代替：

$$L(\omega ,\tilde{\omega},u)=L^{u}(z)log\ [\sigma (v(\tilde{\omega} )^{T}\theta ^{z})](1-L^{u}(z))log\ [1-\sigma (v(\tilde{\omega} )^{T}\theta ^{z})]$$

则似然函数现在写作：

$$L=\sum_{\omega \in C}^{ }\sum_{\tilde{\omega} \in Context(\omega )}^{ }\sum_{u\in \{\omega\}\cup NEG(\omega)}^{ }L(\omega ,\tilde{\omega},u)$$

我们求接这个最优化问题。首先，我们求接该似然函数关于theta的导数，在求解之前首先要补充一个关于sigmod函数求导的公式：

$$[log\ sigmod(x)]^{'}=[1-sigmod(x)]x^{'}$$